In [ ]:
import pandas as pd                                                              # leitura e processamento de datasets
import numpy as np                                                               # processamento matemático em dados
import matplotlib.pyplot as plt                                                  # gerar gráficos
from sklearn.preprocessing import PolynomialFeatures, StandardScaler             # modelos estatísticos para aplicar aos dados
from sklearn.linear_model import LogisticRegression

# Dados simulados com relação não linear
np.random.seed(42)

idade = np.random.randint(20, 70, 20)
renda = np.random.randint(2000, 10000, 20)

# Classe "compra" depende de uma relação curva: idade² + renda/1000 > limiar
compra = ((idade - 40)**2 + renda/1000) > 50
compra = compra.astype(int)


data = pd.DataFrame({
    'idade': idade,
    'renda': renda,
    'compra': compra
})
# Reverter 0/1 para Nao/Sim manualmente
data['compra'] = ['Nao' if x==0 else 'Sim' for x in data['compra']]

print(data)


In [ ]:
X = data[['idade','renda']]
y = data['compra']

# Criar termos polinomiais (idade², renda², idade*renda)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Escalonar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)


In [ ]:
modelo = LogisticRegression(max_iter=1000)  # max_iter maior evita ConvergenceWarning
modelo.fit(X_scaled, y)


In [ ]:
# Limites do gráfico
x_min, x_max = X['idade'].min() - 5, X['idade'].max() + 5
y_min, y_max = X['renda'].min() - 500, X['renda'].max() + 500

xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

# Transformar meshgrid com polinômios e escalonamento
grid = np.c_[xx.ravel(), yy.ravel()]
grid_poly = poly.transform(grid)
grid_scaled = scaler.transform(grid_poly)

# Prever probabilidades de "Sim"
Z_prob = modelo.predict_proba(grid_scaled)[:,1]
Z_prob = Z_prob.reshape(xx.shape)


In [ ]:
plt.figure(figsize=(10,6))

# Gradiente de probabilidades
contour = plt.contourf(xx, yy, Z_prob, 20, cmap='RdYlBu', alpha=0.5)
plt.colorbar(contour, label='Probabilidade de compra (Sim)')

# Linha de decisão (probabilidade = 0.5)
contour_line = plt.contour(xx, yy, Z_prob, levels=[0.5], colors='green', linewidths=2, linestyles='--')

# Pontos reais
cores = {'Nao':'red', 'Sim':'blue'}
for classe in data['compra'].unique():
    subset = data[data['compra']==classe]
    plt.scatter(subset['idade'], subset['renda'],
                color=cores[classe], label=f'Classe {classe}',
                s=100, edgecolor='k')

plt.xlabel('Idade')
plt.ylabel('Renda')
plt.title('Regressão Logística Polinomial: Probabilidade de Compra')
plt.show()
